In [1]:
%matplotlib widget

In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray.plot as xplt
import cftime
import cartopy.feature as cfeat

from IPython import display
# plt.rcParams['animation.ffmpeg_path'] = '/home/henry/anaconda3/envs/downscaling/bin/ffmpeg'

In [3]:
cp_model_rotated_pole = ccrs.RotatedPole(pole_longitude=177.5, pole_latitude=37.5)
platecarree = ccrs.PlateCarree()

In [4]:
cpmdata = xr.open_mfdataset("../../../data/2.2km/rcp85/01/pr/*.nc").loc[dict(ensemble_member=1, time=slice("1980-12-01","1982-11-30"))]
gcmdata = xr.open_mfdataset("../../../data/60km/rcp85/01/*/day/*.nc").loc[dict(ensemble_member=1, time=slice("1980-12-01","1982-11-30"))]
regridded_gcmdata = xr.open_mfdataset('../../../derived_data/60km/rcp85/01/*/day/*.nc').loc[dict(ensemble_member=1, time=slice("1980-12-01","1982-11-30"))]

In [9]:
f, ((ax1, ax2)) = plt.subplots(1, 2, figsize=(14, 6), subplot_kw={'projection': cp_model_rotated_pole})

wettest_itime = gcmdata.sum(dim=["projection_y_coordinate", "projection_x_coordinate"]).pr.argmax(dim="time").values

gcmdata.isel({'time': wettest_itime}).pr.plot(ax=ax1, x='longitude', y='latitude', add_colorbar=True, transform=platecarree, vmin=0, vmax=77)
ax1.set_title("GCM rainfall 1982-11-12")
cpmdata.isel({'time': wettest_itime}).pr.plot(ax=ax2, add_colorbar=True, transform=cp_model_rotated_pole, vmin=0, vmax=77)
ax2.set_title("Local-scale rainfall 1982-11-12")

ax1.coastlines()
ax2.coastlines()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/henry/anaconda3/envs/downscaling/lib/python3.9/site-packages/cartopy/mpl/geoaxes.py:1702: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  X, Y, C, shading = self._pcolorargs('pcolormesh', *args,


In [11]:
import matplotlib.animation as animation



def make_figure():
    fig = plt.figure(figsize=(8, 3))
    ax = fig.add_subplot(1, 1, 1, projection=cp_model_rotated_pole)

    # generate a basemap with country borders, oceans and coastlines
#     ax.add_feature(cfeat.LAND)
#     ax.add_feature(cfeat.OCEAN)
    ax.add_feature(cfeat.COASTLINE)
    return fig, ax

fig,ax = make_figure()


animation_data = cpmdata.sel({"time": slice("1982-11-10", "1982-11-17")})

frames = animation_data.time.size        # Number of frames
min_value = animation_data.pr.values.min()  # Lowest value
max_value = animation_data.pr.values.max()  # Highest value


def draw(frame, add_colorbar):
    grid = animation_data.pr[frame]
    contour = grid.plot(ax=ax, transform=cp_model_rotated_pole, add_colorbar=add_colorbar, vmin=min_value, vmax=max_value)
    title = u"%s — %s" % (animation_data.pr.long_name, str(animation_data.time[frame].values)[:19])
    ax.set_title(title)
    return contour


def init():
    return draw(0, add_colorbar=True)


def animate(frame):
    return draw(frame, add_colorbar=False)


ani = animation.FuncAnimation(fig, animate, frames=frames, interval=500, blit=False,
                              init_func=init, repeat=False)
# plt.show()
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()     
# ax.coastlines()
ani.save('something.gif', writer=animation.FFMpegWriter(fps=8))
# plt.close(fig)

# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …